# Description

# Importing Essential Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('Done')

Done


# Loading Dataset 

In [2]:
path = r'D:\Data_Projects\Password_Strength_Checker\data\raw\data.csv'
data = pd.read_csv(path, on_bad_lines='skip')

# Data Exploration and Analysis

In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
strength,669640.0,0.990196,0.507948,0.0,1.0,1.0,1.0,2.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669640 entries, 0 to 669639
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   password  669639 non-null  object
 1   strength  669640 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [7]:
print(f'There are {data.shape[0]} rows and {data.shape[1]} columns in the dataset')

There are 669640 rows and 2 columns in the dataset


Let's look for rows with only one missing value.

In [16]:
missing_values = data.isnull().sum()
missing_values

password    1
strength    0
dtype: int64

OK! I have only one missing value why?

In [5]:
data[data['password'].isnull()]

,password,strength
367579,NaN,0


I decided to **drop** the row with the missing value because it constitutes a very small fraction of the dataset. Retaining this row would not provide significant insights and could potentially introduce noise into the analysis. By removing this single row, we ensure the integrity and completeness of our dataset, which is crucial for accurate data analysis and modeling.

In [8]:
data.dropna(inplace=True)

## EDA